In [5]:
import os
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
loader = DirectoryLoader("data/eval_documents")
documents = loader.load()

In [7]:
# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

Filename and doc_id are the same for all nodes.               
Generating: 100%|██████████| 10/10 [02:40<00:00, 16.04s/it]


In [9]:
df = testset.to_pandas()
df.to_parquet("data/eval_questions.parquet")
df.to_csv("data/eval_questions.csv")
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the requirement for providing evidence...,[40-2 verbindlich zu berücksichtigen sind.\n\n...,The requirement for providing evidence of equi...,simple,[{'source': 'data/eval_documents/merkblatt_kie...,True
1,"What is the legal definition of the term ""barr...",[40-2 verbindlich zu berücksichtigen sind.\n\n...,The legal definition of the term 'barrierefrei...,simple,[{'source': 'data/eval_documents/merkblatt_kie...,True
2,What are the requirements for ensuring accessi...,[Barrierefreiheit / barrierefreies Bauen\n\nRe...,The requirements for ensuring accessibility an...,simple,[{'source': 'data/eval_documents/merkblatt_kie...,True
3,"What is the legal definition of the term ""barr...",[40-2 verbindlich zu berücksichtigen sind.\n\n...,The legal definition of the term 'barrierefrei...,simple,[{'source': 'data/eval_documents/merkblatt_kie...,True
4,What are the requirements for barrier-free con...,[Barrierefreiheit / barrierefreies Bauen\n\nRe...,The requirements for barrier-free construction...,simple,[{'source': 'data/eval_documents/merkblatt_kie...,True
5,What are the dimensions for the movement areas...,[ach § 50 Abs. 1 LBO\n\nbarrierefrei = nicht r...,The dimensions for the movement areas in front...,reasoning,[{'source': 'data/eval_documents/merkblatt_kie...,True
6,What are the requirements for an accessible ba...,[ach § 50 Abs. 1 LBO\n\nbarrierefrei = nicht r...,"According to LBO and DIN-Normen 18040 Teil 2, ...",multi_context,[{'source': 'data/eval_documents/merkblatt_kie...,True
7,What are the technical requirements for barrie...,[ach § 50 Abs. 1 LBO\n\nbarrierefrei = nicht r...,The technical requirements for barrier-free bu...,multi_context,[{'source': 'data/eval_documents/merkblatt_kie...,True
8,What is the significance of Technische Baubest...,[40-2 verbindlich zu berücksichtigen sind.\n\n...,Technische Baubestimmungen must be observed by...,simple,[{'source': 'data/eval_documents/merkblatt_kie...,True


In [10]:
df.iloc[0]["question"]

'What is the requirement for providing evidence of equivalence in relation to the technical building regulations?'

In [11]:
df.iloc[0]["ground_truth"]

'The requirement for providing evidence of equivalence in relation to the technical building regulations is that deviations from the technical building regulations can only be made through compensation of the requirements of the norms. The burden of proof for fulfilling the requirements through compensation with "other measures" lies with the client.'

In [6]:
import requests
import json

def generate_text(prompt):
    url = "http://localhost:11434/api/generate"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "parameters": {
            "max_length": 100
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        response_text = response.text
        # data = json.loads(response_text)
        # actual_response = data["response"]
        return response_text
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
prompt = "Explain the concept of neural networks."
generated_text = generate_text(prompt)
print(generated_text)

{"model":"llama3","created_at":"2024-06-30T15:23:50.979359185Z","response":"The","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.098110705Z","response":" fascinating","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.223675256Z","response":" world","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.342990433Z","response":" of","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.467174557Z","response":" neural","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.586956066Z","response":" networks","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.719888369Z","response":"!\n\n","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.846302928Z","response":"A","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:51.966522507Z","response":" neural","done":false}
{"model":"llama3","created_at":"2024-06-30T15:23:52.098200271Z","response":" network","done":false}
{"model":"llama3","